In [18]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

class TwitterClient(object):
	'''
	Generic Twitter Class for sentiment analysis.
	'''
	def __init__(self):
		'''
		Class constructor or initialization method.
		'''
		# keys and tokens from the Twitter Dev Console
		consumer_key = 'Wc76KZsr3yaentEWwqVETLmvW'
		consumer_secret = 'Bs7KqjM4U30QX7Kd2l82pkVQ6pesj3syhXi6w5whxSWHzPcejK'
		access_token = '1380496082132631554-S9jR3NPNcf76DLrx6IcQhuStTeGVTI'
		access_token_secret = 'H4NYny1uaG5u7cIydE87S3dVEgBH2BFeXj39Pba7xgwqv'

		# attempt authentication
		try:
			# create OAuthHandler object
			self.auth = OAuthHandler(consumer_key, consumer_secret)
			# set access token and secret
			self.auth.set_access_token(access_token, access_token_secret)
			# create tweepy API object to fetch tweets
			self.api = tweepy.API(self.auth)
		except:
			print("Error: Authentication Failed")

	def clean_tweet(self, tweet):
		'''
		Utility function to clean tweet text by removing links, special characters
		using simple regex statements.
		'''
		return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

	def get_tweet_sentiment(self, tweet):
		'''
		Utility function to classify sentiment of passed tweet
		using textblob's sentiment method
		'''
		# create TextBlob object of passed tweet text
		analysis = TextBlob(self.clean_tweet(tweet))
		# set sentiment
		if analysis.sentiment.polarity > 0:
			return 'positive'
		elif analysis.sentiment.polarity == 0:
			return 'neutral'
		else:
			return 'negative'

	def get_tweets(self, query, count = 10):
		'''
		Main function to fetch tweets and parse them.
		'''
		# empty list to store parsed tweets
		tweets = []

		try:
			# call twitter api to fetch tweets
			fetched_tweets = self.api.search_tweets(q = query, count = count)

			# parsing tweets one by one
			for tweet in fetched_tweets:
				# empty dictionary to store required params of a tweet
				parsed_tweet = {}

				# saving text of tweet
				parsed_tweet['text'] = tweet.text
				# saving sentiment of tweet
				parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

				# appending parsed tweet to tweets list
				if tweet.retweet_count > 0:
					# if tweet has retweets, ensure that it is appended only once
					if parsed_tweet not in tweets:
						tweets.append(parsed_tweet)
				else:
					tweets.append(parsed_tweet)

			# return parsed tweets
			return tweets

		except AttributeError as e:
			# print error (if any)
			print("Error : " + str(e))

def main():
	# creating object of TwitterClient Class
	api = TwitterClient()
	# calling function to get tweets
	tweets = api.get_tweets(query = 'ukraine', count = 100)

	# picking positive tweets from tweets
	ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
	ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']

	print('positive : Negative',len(ptweets),':',len(ntweets))
	# percentage of positive tweets
	print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
	# picking negative tweets from tweets
	ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
	# percentage of negative tweets
	print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
	# percentage of neutral tweets
	#print("Neutral tweets percentage: {} % ".format(100*len(tweets - ntweets - ptweets)/len(tweets)))

	# printing first 5 positive tweets
	print("\n\nPositive tweets:")
	for tweet in ptweets[:1000]:
		print(tweet['text'])

	# printing first 5 negative tweets
	print("\n\nNegative tweets:")
	for tweet in ntweets[:1000]:
		print(tweet['text'])

if __name__ == "__main__":
	# calling main function
	main()

positive : Negative 32 : 13
Positive tweets percentage: 36.36363636363637 %
Negative tweets percentage: 14.772727272727273 %


Positive tweets:
RT @BitcoinBasecamp: It would cost $79 billion per year to end tuition at all public universities in the United States. We’ve sent more tha…
RT @The_Real_Fly: DEMOCRATIC UKRAINE 
https://t.co/RnPE3qmBcX
RT @maria_avdv: I asked these children in Kyiv what they would say to the children of the world. They said: I love Ukraine, Glory to the Ar…
RT @Biz_Ukraine_Mag: “It’s better to spend 10 years behind bars as an honest man than quietly burn with shame over the blood spilled by you…
RT @ukraine_world: Berlin repeats that Russian military facilities are legitimate targets for Ukraine's forces due to the right to self-def…
RT @apiontkovsky: Pentagon gives Ukraine green light for drone strikes inside Russia | News | The Times https://t.co/ni9qZIAehp
RT @MyFirstCousin: What with 'watergate', 'coal to Ukraine', 'strategic fuel stocks' and good knows wh

In [2]:
!pip  install tweepy

     |████████████████████████████████| 101 kB 2.5 MB/s ta 0:00:011
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
